In [20]:
from pathlib import Path
import pickle
import time

from shapely import Point

import pyclipper2

SCALE_FACTOR = 1000000  # Convert float to int64 for Clipper


def shapely_to_path64(poly):
    """Convert Shapely polygon to Path64 (list of Point64)"""
    if poly.is_empty:
        return []
    coords = list(poly.exterior.coords)[:-1]  # Remove duplicate last point
    # Create Point64 objects
    return [
        pyclipper2.Point64(int(x * SCALE_FACTOR), int(y * SCALE_FACTOR))
        for x, y in coords
    ]


output = []

path = Path("data/shapes/")

intersecting_shape = Point(0, 0).buffer(4)
intersecting_shape_pc = [shapely_to_path64(intersecting_shape)]

for i in path.iterdir():
    with open(i, "rb") as openfile:
        shape = pickle.load(openfile)
        vertices = len(shape.boundary.coords)

        ### shapely
        start_time = time.perf_counter()
        for i in range(100):
            shape.intersection(intersecting_shape)
        total_time = time.perf_counter() - start_time
        output.append((vertices, "shapely", total_time))

        ### pyclipper
        shape_pc = [shapely_to_path64(shape)]
        start_time = time.perf_counter()
        for i in range(100):
            pyclipper2.intersection(
                shape_pc, intersecting_shape_pc, pyclipper2.FillRule.NON_ZERO
            )
        total_time = time.perf_counter() - start_time
        output.append((vertices, "pyclipper", total_time))

In [21]:
import polars as pl

df = pl.DataFrame(
    output,
    schema={"vertices": pl.Int64, "library": pl.Utf8, "time": pl.Float64},
    orient="row",
)

df = df.sort(by="vertices")

In [ ]:
import plotly.express as px

fig = px.scatter(
    df,
    x="vertices",
    y="time",
    color="library",
    opacity=0.05,
    trendline="lowess",
    trendline_options=dict(frac=0.1),
)
fig.update_layout(yaxis_range=[0, 0.01])
fig.show()